In [21]:
import pandas as pd
import contractions

# Seth Godin's Blog Dataset - EDA

In [22]:
dataset = pd.read_csv('../data/seth-data.csv')

dataset.head()

,Unnamed: 0,id,url,title,content_plain,content_html,stars,publication-date,referral-url
0,0,24220,https://seths.blog/2002/01/free_signed_cop/,Free Signed Copy of SURVIVAL IS NOT ENOUGH,Don’t know if you read the column I wrote abo...,Don’t know if you read the column I wrote abo...,104,"January 17, 2002",https://seths.blog/page/742/
1,1,24221,https://seths.blog/2002/01/free_ebook_buy_/,Free eBook,Buy a copy of the new paperback edition of Th...,Buy a copy of the new paperback edition of Th...,145,NaN,https://seths.blog/page/742/
2,2,24222,https://seths.blog/2002/01/free_quiz_find_/,Free Quiz,Find out if you zoom. My friend Red did an am...,Find out if you zoom. My friend Red did an am...,112,NaN,https://seths.blog/page/742/
3,3,24223,https://seths.blog/2002/01/thanks_survival/,Thanks,Survival is Not Enough. hit #4 on the Denver ...,Survival is Not Enough. hit #4 on the Denver ...,140,NaN,https://seths.blog/page/742/
4,4,24224,https://seths.blog/2002/01/boring_i_was_st/,Boring,I was stranded today. I had to spend hours in ...,I was stranded today. I had to spend hours in ...,618,"January 15, 2002",https://seths.blog/page/742/


In [23]:
dataset = dataset[dataset['content_plain'].notna()]

dataset = dataset[['content_plain']]

## Data Cleaning

In [24]:
dataset['content_plain'] = dataset['content_plain'].str.lower()
dataset['content_plain'] = dataset['content_plain'].str.replace("'s ", " ")
dataset['content_plain'] = dataset['content_plain'].str.replace("'s\.", ".")
dataset['content_plain'] = dataset['content_plain'].str.replace("'s,", ',')
dataset['content_plain'] = dataset['content_plain'].str.replace("'s:", ':')
#dataset['content_plain'] = dataset['content_plain'].str.replace("'s)", ")")
dataset['content_plain'] = dataset['content_plain'].str.replace('’', " ")
dataset['content_plain'] = dataset['content_plain'].str.replace('‘', " ")
dataset['content_plain'] = dataset['content_plain'].str.replace('“', ' ')
dataset['content_plain'] = dataset['content_plain'].str.replace('”', ' ')
dataset['content_plain'] = dataset['content_plain'].str.replace('"', ' ')
dataset['content_plain'] = dataset['content_plain'].str.replace('\.\.\.', '.')
dataset['content_plain'] = dataset['content_plain'].str.replace('\.\.', '.')
dataset['content_plain'] = dataset['content_plain'].str.replace('…', '.')
dataset['content_plain'] = dataset['content_plain'].str.replace('—', '-')
dataset['content_plain'] = dataset['content_plain'].str.replace('-', ' - ')
dataset['content_plain'] = dataset['content_plain'].str.replace('–', ' - ')
dataset['content_plain'] = dataset['content_plain'].str.replace("™", ' ')
dataset['content_plain'] = dataset['content_plain'].str.replace("\xa0", ' ')
dataset['content_plain'] = dataset['content_plain'].str.replace("\xa02", ' ')
dataset['content_plain'] = dataset['content_plain'].str.replace(" s ", "")

/var/folders/s1/ts3nxvl965lfts126qzdcw300000gn/T/ipykernel_49358/46317691.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset['content_plain'] = dataset['content_plain'].str.replace("'s\.", ".")
/var/folders/s1/ts3nxvl965lfts126qzdcw300000gn/T/ipykernel_49358/46317691.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset['content_plain'] = dataset['content_plain'].str.replace('\.\.\.', '.')
/var/folders/s1/ts3nxvl965lfts126qzdcw300000gn/T/ipykernel_49358/46317691.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset['content_plain'] = dataset['content_plain'].str.replace('\.\.', '.')


In [25]:
dataset.head(20)

,content_plain
0,don t know if you read the column i wrote abo...
1,buy a copy of the new paperback edition of th...
2,find out if you zoom. my friend red did an am...
3,survival is not enough. hit #4 on the denver ...
4,i was stranded today. i had to spend hours in ...
5,working out at the awful marriott outside of t...
6,google s secret hope it s still live by the t...
7,"so there it is, in the fine print at the botto..."
8,"lately, the entertainment mega - companies app..."
9,"the editors at inside.com (now, alas, basicall..."


In [26]:
dataset['content_plain'] = dataset['content_plain'].apply(lambda post: contractions.fix(post))

In [27]:
dataset.to_csv('../data/clean_dataset.csv', index=False)